#### Announcement of AI tool usage:  
#### in Part 2 of this notebook, this project was inspired by Chatgpt in how to solve the problem of "unneeded information (like runway or time) if there is no delay happening"

#### Part 0: preparations and reading data  
##### **please unzip the file data.rar to read the data**

In [10]:
import requests
import pandas as pd
from pprint import pprint
import config
from config import api
import os
from io import StringIO

# Specify the folder path
folder_path = '../Data/data.csv'

df = pd.read_csv(folder_path)

#### Part 1: Filtering initial data source to have Airport IATA  
##### Extract the information from 'airport' to the 1st comma

In [11]:
df['departure airport IATA'] = df['departure'].apply(lambda x: x.split(',')[0][1:-1])
df['arrival airport IATA'] = df['arrival'].apply(lambda x: x.split(',')[0][1:-1])
airport_IATA_df = df[['departure airport IATA', 'arrival airport IATA']]

# Save file to Data Folder
folder_name = 'Data'
folder_path = os.path.join(os.pardir, folder_name)
file_path = os.path.join(folder_path, 'Airport_IATA.csv')

airport_IATA_df.to_csv(file_path, index=False)

#### Part 2: Filtering initial data source to have Delay Time  
##### Extract the information starting from the 4th comma, all the way to the 5th comma, which is the info for "actual delay"

In [12]:
df['delay in mins'] = df['departure'].apply(lambda x: (x.split(',')[4:5] + [None])[0] if x else None)
# substitude info with 'delay': N/A if no delay info is detected
df['delay in mins'] = df['delay in mins'].apply(lambda info: 'None' if info is None or ('Time' in info or 'Runway' in info) else info)

delay_df = df['delay in mins']

folder_name = 'Data'
folder_path = os.path.join(os.pardir, folder_name)
file_path = os.path.join(folder_path, 'Delay.csv')

delay_df.to_csv(file_path, index=False)

#### Code to collect the estimated delay: subtracting estimated Time by scheduled Time

In [14]:
# Function to extract scheduled time from departure details
def extract_scheduled_time(row):
    departure_details = eval(row['departure'])
    return pd.to_datetime(departure_details.get('scheduledTime', pd.NaT))

# Function to extract estimated time from departure details
def extract_estimated_time(row):
    departure_details = eval(row['departure'])
    return pd.to_datetime(departure_details.get('estimatedTime', pd.NaT))

# Extract relevant information using the defined functions
df['scheduledTime'] = df.apply(extract_scheduled_time, axis=1)
df['estimatedTime'] = df.apply(extract_estimated_time, axis=1)
df['estimatedDelay'] = (df['estimatedTime'] - df['scheduledTime']).dt.total_seconds() / 60.0
estimated_delay_df = df['estimatedDelay']

folder_name = 'Data'
folder_path = os.path.join(os.pardir, folder_name)
file_path = os.path.join(folder_path, 'Estimated_Delay.csv')
estimated_delay_df.to_csv(file_path, index=False)


#### Part 3: Filtering initial data source to have Airline Companies
##### Extract the information in column "airline" until it meets the 1st comma

In [13]:
df['Airline Company'] = df['airline'].apply(lambda x: x.split(',')[0][1:-1])
airline_df = df['Airline Company']

folder_name = 'Data'
folder_path = os.path.join(os.pardir, folder_name)
file_path = os.path.join(folder_path, 'Airline_Companies.csv')

airline_df.to_csv(file_path, index=False)

#### Part 4: filtering data to have Date Information

In [15]:
df['scheduledTime'] = df['scheduledTime'].astype(str)
df['month_day'] = df['scheduledTime'].str.extract(r'-(\d{2}-\d{2})')
date_df = df['month_day']

folder_name = 'Data'
folder_path = os.path.join(os.pardir, folder_name)
file_path = os.path.join(folder_path, 'Date_Info.csv')

date_df.to_csv(file_path, index=False)

#### Part 5: Create a new DF that merges all info attained above

In [16]:
final_df = df[['departure airport IATA', 'arrival airport IATA', 'delay in mins', 'estimatedDelay', 'Airline Company', 'month_day']]

folder_name = 'Final Data'
folder_path = os.path.join(os.pardir, folder_name)
file_path = os.path.join(folder_path, 'final_data.csv')
final_df.to_csv(file_path, index=False)